In [1]:
# Headers
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
from copy import deepcopy
from skimage.feature import greycomatrix, greycoprops
import cv2  
from numpy.linalg import det
from skimage import util, exposure
from math import sqrt, ceil
from skimage.segmentation import slic
from skimage.segmentation import mark_boundaries
from skimage.util import img_as_float
from skimage import io

In [2]:
# global variables
black_value = np.float64(-1408.5106382978724)
images_filename = 'tr_im.nii'
masks_filename = 'tr_mask.nii'

In [3]:
def get_vals(mask):
    vals = []
    x, y = mask.shape
    for i in range(x):
        for j in range(y):
            if mask[i,j] not in vals:
                vals.append(mask[i][j])

    vals.remove(0)
    return vals

In [4]:
def apply_mask(img, mask, class_id):
    heigh, width = img.shape
    tmp = deepcopy(img)
    lineal_array = np.array(0)
    for x in range(heigh):
        for y in range(width):
            # Set pixel as black color if is not inside this mask
            if mask[x,y] != np.float64(class_id):
                tmp[x,y] = black_value
            # Count pixels inside this mask
            else:
                lineal_array = np.append(lineal_array, tmp[x,y])
    # In tmp is saved the image with mask applied
    # In lineal_array is saved just the mask's pixels
    lineal_array = np.delete(lineal_array, 0)
    return tmp, lineal_to_matrix(lineal_array)

In [5]:
def calculate_media_lineal_arr(array):
    length = array.shape[0]
    media = np.float64(0)
    for x in range(length):
        media += array[x]
    media = media/length
    return media

In [6]:
def lineal_to_matrix(lineal_array):
    sqrt_ = ceil(sqrt(lineal_array.shape[0]))
    media = calculate_media_lineal_arr(lineal_array)
    missing_values = sqrt_**2 - lineal_array.shape[0]
    # Complete square matrix
    for x in range(missing_values):
        lineal_array = np.append(lineal_array, media)
    
    return lineal_array.reshape([sqrt_, sqrt_])

In [7]:
def show_slice_mask(slicei, mask):
    """ Function to display images and mask """
    fig, ax = plt.subplots(1,2)
    ax[0].imshow(slicei.T,  cmap="gray", origin="lower")
    ax[0].set_title('Image')
    ax[1].imshow(mask.T, cmap="gray", origin="lower")
    ax[1].set_title('Mask')
    plt.show()

In [8]:
def glcm_properties(image):
    distancias = [1,2,3]
    #0 -> Oeste a este, np.pi/4 -> diagonal hacia abajo, np.pi/2 -> Norte a sur
    angulos = [0, np.pi/4, np.pi/2]

    # Calculate properties for just one superpixel using the
    # respective mask  
    glcm = greycomatrix(image, 
                        distances = distancias, 
                        angles = angulos,
                        symmetric=True, 
                        normed=True)
    energyResults = greycoprops(glcm, 'energy')
    #homogeneityResults = greycoprops(glcm, 'homogeneity')
    contrastResults = greycoprops(glcm, 'contrast')
    correlationResults = greycoprops(glcm, 'correlation')
    #dissimilarityResults = greycoprops(glcm, 'dissimilarity')
    
    return [contrastResults, correlationResults, energyResults]

In [9]:
def training(imgs, masks):
    height, width, num_images = imgs.shape
    # Class dictionary to store every property for this class as key
    classes_dict = {}
    # loop over the images
    for x in range(num_images):
        # Get every class in the current image
        classes = get_vals(masks[:,:,x])
        # loop for every class in the current image 
        for j in classes:
            if not int(j) in classes_dict:
                classes_dict[int(j)] = [[np.zeros((3,3)), 0], [np.zeros((3,3)), 0], [np.zeros((3,3)), 0]]
            # getting the square matrix for the current class
            tmp, matrix_mask = apply_mask(imgs[:,:,x], masks[:,:,x], j)
            # Contrast, Correlation, Energy properties
            props = glcm_properties(matrix_mask.astype(np.uint8))
            # Adding every class's property values
            for z in range(len(props)):
                classes_dict[int(j)][z][0] += props[z]
                classes_dict[int(j)][z][1] += 1
        
        if x+1 % 10 == 0:
            print("{} imagenes procesadas".format(x))

    # Getting average for every class's property
    for id_ in classes_dict.keys():
        for z in range(3):
            classes_dict[id_][z][0] = classes_dict[id_][z][0]/classes_dict[id_][z][1]
            classes_dict[id_][z] = classes_dict[id_][z][0]
    #0: correlation, 1: contrast, 2: energy
    return classes_dict

In [10]:
def main():
    to_show = 3
    imgs = nib.load(images_filename).get_fdata()
    masks = nib.load(masks_filename).get_fdata()
    train = training(imgs, masks)
    print(train)
    """
    #for x in range(3):
    for x in [25, 26, 27]:
        print("Image no ", x)
        classes = get_vals(masks[:,:,x])
        show_slice_mask(imgs[:,:,x], masks[:,:,x])
        for j in classes:
            tmp, matrix_mask = apply_mask(imgs[:,:,x], masks[:,:,x], j)
            print("class ", j)
            print(type(matrix_mask))
            #show_slice_mask(tmp, masks[:,:,x])
            show_slice_mask(tmp, matrix_mask)
            print(glcm_properties(matrix_mask.astype(np.uint8)))
            #create_matrix(tmp)
        #print(get_prop(imgs[:,:,x].astype(np.uint8), masks[:,:,x].astype(np.uint8)))
        #get_every_mask(imgs[:,:,x], masks[:,:,x])
    """

In [11]:
main()

{2: [array([[ 9385.37065772, 11163.47052478, 11186.92940868],
       [10420.23405423, 11163.47052478, 11394.38585975],
       [10775.21249353, 11391.17985803, 11446.06550944]]), array([[ 0.1671462 ,  0.01764346,  0.01558263],
       [ 0.07896085,  0.01764346, -0.00284327],
       [ 0.04724605, -0.00347878, -0.0066721 ]]), array([[0.03293342, 0.02107843, 0.02094445],
       [0.03231313, 0.02107843, 0.01852209],
       [0.03202098, 0.01920837, 0.01882293]])], 1: [array([[ 8532.47988509, 10981.95066758, 10982.2624071 ],
       [10133.29681999, 10981.95066758, 11140.99894608],
       [10550.07887752, 11088.86357162, 11139.81834338]]), array([[0.23510234, 0.01584693, 0.01607827],
       [0.0922434 , 0.01584693, 0.00178963],
       [0.05505664, 0.00646262, 0.00187555]]), array([[0.01731996, 0.01172741, 0.01167576],
       [0.01682207, 0.01172741, 0.0108741 ],
       [0.01669869, 0.01099871, 0.01092407]])], 3: [array([[10426.67756157, 12163.44708504, 12209.70068343],
       [11870.50027707, 1